In [1]:
import torch
import numpy as np
from torchvision.transforms import transforms
import random
from PIL import Image
from resnet18 import ResNet18
from torch.autograd import Variable
from resnet18_inc_v2 import IncrementalResNet18V2
import torch.utils.model_zoo as model_zoo

from inception3 import Inception3, Inception3My

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
batch_size = 1
patch_size = 4
input_size = 299

image_patch = torch.cuda.FloatTensor(3, patch_size, patch_size).fill_(0)

x_loc = random.sample(range(0, input_size - patch_size), batch_size)
y_loc = random.sample(range(0, input_size - patch_size), batch_size)
patch_locations = zip(x_loc, y_loc)
patch_locations = [(101, 92)]

loader = transforms.Compose([transforms.Resize([299, 299]), transforms.ToTensor()])
images1 = Image.open('./dog_resized.jpg')
images1 = loader(images1)
images1 = images1.unsqueeze(0)
images1 = images1.repeat(batch_size, 1, 1, 1).cuda()
#images2 = images1.clone()

inc_model = Inception3().eval().cuda()
inc_model.load_state_dict(model_zoo.load_url('https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth'))
inc_model.eval()
model = Inception3My().eval()
model.eval()

for i,(x,y) in enumerate(patch_locations):
    images1[i, :, x:x+patch_size, y:y+patch_size] = image_patch

y = model(images1)
#y = inc_model.Mixed_5b(y)


patch_locations = torch.from_numpy(np.array(patch_locations, dtype=np.int32))
x = inc_model.forward(images1)


# y = model.conv1_op[1].weight.data
# x = inc_model.Conv2d_1a_3x3.bn.weight.data

temp = (y-x).cpu().data.numpy()
# import matplotlib.pyplot as plt
# plt.imshow(temp[0,225,:,:])
# plt.colorbar()
# plt.show()
print(np.sum(np.abs(temp)))

0.0
